<a href="https://colab.research.google.com/github/aleksandralola/memeometer/blob/main/MemoMeter_Augmanted_Data_Model_FV_Clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing packages and importing libraries

In [ ]:
# Installing required packages
!pip install gradio pandas numpy matplotlib scikit-learn opencv-python pillow tf_keras pytesseract transformers tensorflow albumentations

In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade keras
!pip install --upgrade transformers

In [ ]:
# Importing required libraries
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from transformers import DistilBertTokenizer, TFDistilBertModel

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from huggingface_hub import login
login()  # Will prompt for token interactively if run in notebook

# Uploading data

Dataset has been already augmented in another notebook and uploaded here after augmenting.

In [ ]:
# Linking dataset paths
drive_path = '/content/drive/MyDrive/'
dataset_path = drive_path + 'memotion_dataset_7k/labels_augmented_v2.csv'
images_folder_path = drive_path + 'memotion_dataset_7k/images_aug/'

In [ ]:
# Loading dataset
try:
    df = pd.read_csv(dataset_path)
    print("Dataset columns:", df.columns.tolist())
except Exception as e:
    print(f"Error loading dataset: {e}")
    raise

In [ ]:
# Verifying image paths
df['image_path'] = images_folder_path + df['image_name']
print("\nSample image paths:")
print(df['image_path'].head())

# Checking data distribution

In [ ]:
# Specifying columns to analyze
columns_to_plot = ['humour', 'sarcasm', 'offensive', 'motivational', 'overall_sentiment']

# Create subplots with 3 charts per row
fig, axes = plt.subplots(nrows=(len(columns_to_plot) + 2) // 3, ncols=3, figsize=(15, 10))  # Adjust the figure size
axes = axes.flatten()  # Flatten axes array for easy indexing

# Calculating and plotting the distribution of values in each specified column
for i, column in enumerate(columns_to_plot):
    if column in df.columns:
        value_counts = df[column].value_counts()

        # Plotting the distribution on the corresponding subplot
        value_counts.plot(kind='bar', color='skyblue', ax=axes[i])
        axes[i].set_title(f'Distribution of values in "{column}"')
        axes[i].set_xlabel('Values')
        axes[i].set_ylabel('Frequency')
        axes[i].tick_params(axis='x', rotation=45)
    else:
        print(f"Column '{column}' not found in the dataset.")
        axes[i].axis('off')  # Turn off unused subplot if column is missing

# Hide unused subplots if any
for j in range(len(columns_to_plot), len(axes)):
    axes[j].axis('off')  # Turn off unused subplots

plt.tight_layout()
plt.show()


# Building preprocessing functions

In [ ]:
# Text preprocessing function

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def preprocess_text(texts, max_length=128):
    return tokenizer(
        texts.tolist(),
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='tf'
    )

In [ ]:
# Image preprocessing function (with TensorFlow-native error handling)

def safe_load(image_path):
    try:
        image = tf.io.read_file(image_path)
        image = tf.image.decode_image(image, channels=3, expand_animations=False)
        image = tf.image.resize(image, [224, 224])
        image = tf.keras.applications.efficientnet.preprocess_input(image)
        return image
    except:
        # Return a zero-initialized tensor of the same shape instead of None
        return tf.zeros((224, 224, 3), dtype=tf.float32)

In [ ]:
# Ensuring image_paths and df are aligned
valid_indices = []
for i, image_name in enumerate(df['image_name']):
    image_path = os.path.join(images_folder_path, image_name)
    if os.path.exists(image_path):
        valid_indices.append(i)

# Filtering dataframe to include only valid indices
df = df.iloc[valid_indices].reset_index(drop=True)

# Rebuilding image_paths to match the filtered dataframe
image_paths = [os.path.join(images_folder_path, name) for name in df['image_name']]

# Preprocessing images

In [ ]:
# Verifying TensorFlow installation
print("TensorFlow version:", tf.__version__)

# Verify image directory exists
assert os.path.exists(images_folder_path), f"Directory {images_folder_path} not found!"

# Creating image paths and verifying them
image_paths = [os.path.join(images_folder_path, name) for name in df['image_name']]

print("\nSample image paths:")
for path in image_paths[:5]:
    print(f"Exists: {os.path.exists(path)} → {path}")

In [ ]:
# Preprocessing images
def preprocess_image(image):
    """Ensure the image is RGB, even if it's grayscale"""
    # Convert grayscale images to RGB by repeating the channels
    if image.shape[-1] == 1:  # Grayscale image
        image = tf.image.grayscale_to_rgb(image)
    return image

def safe_load(image_path):
    """Load and preprocess image with error handling"""
    try:
        # Read and decode image
        image = tf.io.read_file(image_path)
        image = tf.image.decode_image(image, channels=3, expand_animations=False)  # Decode as RGB
        image = tf.image.resize(image, [224, 224])  # Resize to target dimensions
        image = preprocess_image(image)  # Ensure RGB format
        return tf.keras.applications.efficientnet.preprocess_input(image)
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return tf.zeros((224, 224, 3), dtype=tf.float32)  # Return empty image in case of error

# Creating image dataset
image_ds = tf.data.Dataset.from_tensor_slices(image_paths).map(
    lambda path: tf.py_function(safe_load, [path], Tout=tf.float32),
    num_parallel_calls=tf.data.AUTOTUNE
)

# Optionally suppress warnings if they're not critical
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow warnings

# Verifying the dataset
for image in image_ds.take(1):
    print(image.shape)  # Verify the shape of the images after preprocessing

# Preprocessing text

In [ ]:
# Removing rows with NaN or empty strings in 'text_corrected'
df = df.dropna(subset=['text_corrected'])
df = df[df['text_corrected'] != ""]

# Converting all entries to strings (in case of numeric/None values)
df['text_corrected'] = df['text_corrected'].astype(str)

In [ ]:
# Preprocessing text
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def preprocess_text(texts, max_length=128):
    return tokenizer(
        texts.tolist(),
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='tf'
    )

# Tokenizing text
tokenized_texts = preprocess_text(df['text_corrected'])

# Creating text dataset
text_ds = tf.data.Dataset.from_tensor_slices({
    'input_ids': tokenized_texts['input_ids'],
    'attention_mask': tokenized_texts['attention_mask']
})

In [ ]:
# Veryfying images vs text and removing not matching ones
extra_images = []
for image_path in image_paths:
    image_name = os.path.basename(image_path)
    if image_name not in df['image_name'].values:
        extra_images.append(image_path)

filtered_image_paths = [path for path in image_paths if path not in extra_images]
image_paths = filtered_image_paths

# Verifying alignment
print(f"Final image dataset size: {len(image_paths)}")
print(f"Text dataset size: {len(df)}")
assert len(image_paths) == len(df), "Mismatch resolved!"

# Creating label dataset

In [ ]:
# Defining mappings for categorical columns
humour_mapping = {'not_funny': 0, 'funny': 1, 'very_funny': 2, 'hilarious': 3}
sarcasm_mapping = {'not_sarcastic': 0, 'general': 1, 'twisted_meaning': 2,}
offensive_mapping = {'not_offensive': 0, 'slight_offensive':1, 'very_offensive':2,}
motivational_mapping = {'not_motivational': 0, 'motivational': 1}
overall_sentiment_mapping = {'negative': 0, 'neutral': 1, 'positive': 2, 'very_positive': 3}

# Replaceing string values with numerical equivalents using the mappings
# Filling NaN values with a default category (e.g., 'not_sarcastic' for sarcasm)
df['humour'] = df['humour'].map(humour_mapping).fillna(0).astype(int)
df['sarcasm'] = df['sarcasm'].map(sarcasm_mapping).fillna(0).astype(int)
df['offensive'] = df['offensive'].map(offensive_mapping).fillna(0).astype(int)
df['motivational'] = df['motivational'].map(motivational_mapping).fillna(0).astype(int)
df['overall_sentiment'] = df['overall_sentiment'].map(overall_sentiment_mapping).fillna(0).astype(int)

# Creating the labels dataset with the mapped numerical values
labels_ds = tf.data.Dataset.from_tensor_slices({
    'humour': df['humour'].values,
    'sarcasm': df['sarcasm'].values,
    'offensive': df['offensive'].values,
    'motivational': df['motivational'].values,
    'overall_sentiment':df['overall_sentiment'].values
})

# Combining datasets

In [ ]:
def ensure_shapes(example_inputs, example_labels):
    example_inputs['image_input'] = tf.ensure_shape(example_inputs['image_input'], [224, 224, 3])
    return example_inputs, example_labels

# Combining image, text and label datasets
full_ds = tf.data.Dataset.zip((text_ds, image_ds, labels_ds)).map(
    lambda text, img, labels: (
        {'input_ids': text['input_ids'], 'attention_mask': text['attention_mask'], 'image_input': img},
        {'humour': labels['humour'], 'sarcasm': labels['sarcasm'], 'offensive': labels['offensive'], 'motivational': labels['motivational'], 'overall_sentiment': labels['overall_sentiment']}
    )
).map(ensure_shapes)  # Applying the ensure_shapes function to fix image input shapes

# Shuffle and split the dataset

In [ ]:
# Shuffling datasets and splitting into train, validate, and test parts
dataset_size = len(df)
train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)

buffer_size = 9000  # Adjust this value as needed

full_ds = full_ds.shuffle(buffer_size=buffer_size)
train_ds = full_ds.take(train_size)
remaining = full_ds.skip(train_size)
val_ds = remaining.take(val_size)
test_ds = remaining.skip(val_size)

# Batch and optimize the datasets

In [ ]:
# Batching the dataset and optimising
BATCH_SIZE = 64
train_ds = train_ds.cache().batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.cache().batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE)

# Verifying batch structure


In [ ]:
# Verifying batch structure
for batch in train_ds.take(1):
    inputs, labels = batch
    print("Input IDs shape:", inputs['input_ids'].shape)  # (8, 128)
    print("Image shape:", inputs['image_input'].shape)    # (8, 224, 224, 3)
    print("Humour labels shape:", labels['humour'].shape)   # (8,)

# Building the model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model, regularizers
from transformers import TFDistilBertModel

# Defining constants
TEXT_SEQ_LENGTH = 128
IMAGE_SIZE = 224

# Custom DistilBERT wrapper
class DistilBertEncoder(tf.keras.layers.Layer):
    def __init__(self, model_name='distilbert-base-uncased', **kwargs):
        super().__init__(**kwargs)
        self.bert = TFDistilBertModel.from_pretrained(model_name)
        self.bert.trainable = False  # Set to True to fine-tune

    def call(self, inputs):
        input_ids, attention_mask = inputs
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return output.last_hidden_state[:, 0, :]  # [CLS] token


In [ ]:
# Text Input
input_ids = layers.Input(shape=(TEXT_SEQ_LENGTH,), dtype=tf.int32, name='input_ids')
attention_mask = layers.Input(shape=(TEXT_SEQ_LENGTH,), dtype=tf.int32, name='attention_mask')
text_features = DistilBertEncoder()([input_ids, attention_mask])

# Image Input
image_input = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name='image_input')
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)
)
base_model.trainable = False
image_features = layers.GlobalAveragePooling2D()(base_model(image_input))

# Combining inputs
combined = layers.Concatenate()([text_features, image_features])
x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01))(combined)
x = layers.BatchNormalization()(x)
# 👇 No dropout here

# Defining outputs
outputs = {
    'humour': layers.Dense(4, activation='softmax', name='humour')(x),
    'sarcasm': layers.Dense(3, activation='softmax', name='sarcasm')(x),
    'offensive': layers.Dense(3, activation='softmax', name='offensive')(x),
    'motivational': layers.Dense(2, activation='softmax', name='motivational')(x),
    'overall_sentiment': layers.Dense(4, activation='softmax', name='overall_sentiment')(x)
}

# Final model
model = Model(inputs=[input_ids, attention_mask, image_input], outputs=outputs)

In [ ]:
model.compile(
    optimizer='adam',
    loss={
        'humour': 'sparse_categorical_crossentropy',
        'sarcasm': 'sparse_categorical_crossentropy',
        'offensive': 'sparse_categorical_crossentropy',
        'motivational': 'sparse_categorical_crossentropy',
        'overall_sentiment': 'sparse_categorical_crossentropy'
    },
    metrics={
        'humour': 'accuracy',
        'sarcasm': 'accuracy',
        'offensive': 'accuracy',
        'motivational': 'accuracy',
        'overall_sentiment': 'accuracy'
    }
)

# Training and saving the model

In [ ]:
# Training the model with basic settings and 8 epochs
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=8
)

# Saving the entire model
model.save("/content/drive/MyDrive/AI_Models/meme_final_model_aug.keras")

# Evaluating the model with confusion matrix

In [ ]:
# Getting true labels and predicted labels
y_true = {label: [] for label in ['humour', 'sarcasm', 'offensive', 'motivational', 'overall_sentiment']}
y_pred = {label: [] for label in ['humour', 'sarcasm', 'offensive', 'motivational', 'overall_sentiment']}

label_names = ['humour', 'sarcasm', 'offensive', 'motivational', 'overall_sentiment']

for images, labels in test_ds:
    preds = model.predict(images)
    # Access labels and predictions using keys instead of indices
    for label_name in label_names:
        y_true[label_name].extend(labels[label_name].numpy())
        # Access predictions using the label name directly
        y_pred[label_name].extend(np.argmax(preds[label_name], axis=1))

# Evaluating each class
for label in label_names:
    print(f"--- {label.upper()} ---")
    print(classification_report(y_true[label], y_pred[label]))

    cm = confusion_matrix(y_true[label], y_pred[label])
    plt.figure(figsize=(4, 3))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{label.capitalize()} Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

In [ ]:
# Evaluation metrics without the confusion matrix

from sklearn.metrics import classification_report
import numpy as np

# Getting true labels and predicted labels
y_true = {label: [] for label in ['humour', 'sarcasm', 'offensive', 'motivational', 'overall_sentiment']}
y_pred = {label: [] for label in ['humour', 'sarcasm', 'offensive', 'motivational', 'overall_sentiment']}

label_names = ['humour', 'sarcasm', 'offensive', 'motivational', 'overall_sentiment']

for images, labels in test_ds:
    preds = model.predict(images)
    for label_name in label_names:
        y_true[label_name].extend(labels[label_name].numpy())
        y_pred[label_name].extend(np.argmax(preds[label_name], axis=1))

# Evaluating each class and printing metrics
for label in label_names:
    print(f"--- {label.upper()} ---")
    print(classification_report(y_true[label], y_pred[label], digits=4))
    print("\n" + "="*50 + "\n")

In [ ]:
# Saving the entire model again (just in case :)
model.save("/content/drive/MyDrive/AI_Models/meme_final_model_aug3.keras")

# Gradio Prototyping

In [ ]:
!pip install gradio pytesseract

In [ ]:
# Installing Tesseract-OCR
!sudo apt-get update
!sudo apt-get install -y tesseract-ocr

In [ ]:
import gradio as gr
import pytesseract
from PIL import Image

pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [ ]:
# Testing OCR with a sample image
def test_ocr():
    img = Image.open("/content/drive/MyDrive/memotion_dataset_7k/images/image_333.jpg")
    text = pytesseract.image_to_string(img)
    print("Extracted Text:", text)

test_ocr()

In [ ]:
import numpy as np
import tensorflow as tf
from transformers import DistilBertTokenizer
from PIL import Image
import pytesseract
import gradio as gr

# Loading tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Classifying labels
class_names = {
    'humour': ['😐 Not Funny', '🙂 Funny', '😆 Very Funny', '🤣 Hilarious'],
    'sarcasm': ['🧐 Not Sarcastic', '😏 General Sarcasm', '🤨 Twisted Meaning'],
    'offensive': ['✅ Not Offensive', '⚠️ Slightly Offensive', '🚫 Offensive'],
    'motivational': ['🙃 Not Motivational', '💡 Motivational'],
    'overall_sentiment': ['🙁 Negative', '😐 Neutral', '🙂 Positive', '🌟 Very Positive']
}

# OCR function
def extract_text_from_image(img):
    try:
        gray_img = img.convert('L')
        text = pytesseract.image_to_string(gray_img)
        return text.strip()
    except Exception as e:
        return f"OCR Error: {str(e)}"

# Text preprocessing
def preprocess_text(text):
    return tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='tf'
    )

# Main prediction function
def predict_meme(input_image, user_text=None):
    try:
        image_text = extract_text_from_image(input_image)
        full_text = f"{image_text} {user_text}" if user_text else image_text

        img = input_image.resize((224, 224))
        img_array = np.array(img) / 255.0
        if img_array.shape[-1] != 3:
            img_array = img_array[..., :3]
        img_array = img_array.reshape(1, 224, 224, 3)

        text_encoded = preprocess_text([full_text])

        predictions = model.predict([
            text_encoded['input_ids'],
            text_encoded['attention_mask'],
            img_array
        ])

        # Use correct unpacking
        humour_pred = predictions['humour']
        sarcasm_pred = predictions['sarcasm']
        offensive_pred = predictions['offensive']
        motivational_pred = predictions['motivational']
        overall_sentiment_pred = predictions['overall_sentiment']

        # Debug print
        print("Humour prediction:", humour_pred)
        print("Sarcasm prediction:", sarcasm_pred)
        print("Offensive prediction:", offensive_pred)
        print("Motivational prediction:", motivational_pred)
        print("Overall sentiment prediction:", overall_sentiment_pred)

        # Get highest-probability index for each
        humour_idx = np.argmax(humour_pred[0])
        sarcasm_idx = np.argmax(sarcasm_pred[0])
        offensive_idx = np.argmax(offensive_pred[0])
        motivational_idx = np.argmax(motivational_pred[0])
        overall_sentiment_idx = np.argmax(overall_sentiment_pred[0])

        return (
            class_names['humour'][humour_idx],
            class_names['sarcasm'][sarcasm_idx],
            class_names['offensive'][offensive_idx],
            class_names['motivational'][motivational_idx],
            class_names['overall_sentiment'][overall_sentiment_idx]
        )

    except Exception as e:
        return str(e), "", "", "", ""

# Gradio UI
iface = gr.Interface(
    fn=predict_meme,
    inputs=[
        gr.Image(type="pil", label="Upload Meme Image")
    ],
    outputs=[
        gr.Label(label="Humour"),
        gr.Label(label="Sarcasm"),
        gr.Label(label="Offensive"),
        gr.Label(label="Motivational"),
        gr.Label(label="Overall Sentiment")
    ],
    title="🧠 Meme-o-Meter: AI Sentiment Classifier for Memes",
    description="Upload a meme and receive predictions across 5 emotional dimensions."
)

iface.launch(share=True)
